In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

* Read pickle file

In [2]:
df = pd.read_pickle("df.pkl")
df = df.drop_duplicates()
df = df.query("airport != 'PSC'").reset_index(drop = True)

• Which airport tends to have the greatest delays?


In [3]:
df['total_delay'] = df[[x for x in df.columns if "delay" in x]].sum(axis = 1)

<ipython-input-3-e6fa8e326981>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['total_delay'] = df[[x for x in df.columns if "delay" in x]].sum(axis = 1)


In [4]:
# df_grouped = df.groupby(['airport_name','year','month'])
df_grouped = df.groupby(['airport_name'])
airport_with_most_delay = df_grouped.agg(value = ("total_delay","sum")).sort_values(by=["value"],ascending=False).index.tolist()[0]
value_with_most_delay   = df_grouped.agg(value = ("total_delay","sum")).sort_values(by=["value"],ascending=False).value.tolist()[0]
print(f'The airport with the most delay time is {airport_with_most_delay} and that time is in total for all the months of analysis {value_with_most_delay/3600} hours.')

The airport with the most delay time is Dallas/Fort Worth International and that time is in total for all the months of analysis 4067.293888888889 hours.


### Groupby Descriptive and Aggregattions

* Descriptive

In [5]:
# Function
def category_descriptive(df,group_column,value_column):
    temp_df = df[[group_column,value_column]].reset_index(drop=True)
    temp_df_desc = temp_df.groupby(group_column).describe().reset_index()
    print(f'The Descriptives about "{value_column}" grouped by "{group_column}" are: ')
    display(temp_df_desc)

In [6]:
group_column = 'quarter'
value_column = 'total_delay'
category_descriptive(df,group_column,value_column)

The Descriptives about "total_delay" grouped by "quarter" are: 


quarter total_delay                                                         \
                count         mean           std  min    25%     50%     75%   
0       1      9726.0  5899.131914  17889.551039  0.0  472.0  1396.0  3858.0   
1       2     10083.0  8255.840424  25069.718355  0.0  621.0  1928.0  5532.0   
2       3      8533.0  9863.574827  28644.685819  0.0  874.0  2396.0  6790.0   
3       4      5153.0  7288.515622  20580.810082  0.0  626.0  1698.0  5040.0   

             
        max  
0  410578.0  
1  611388.0  
2  575396.0  
3  373714.0

In [7]:
group_column = 'semester'
value_column = 'arrival_delay'
category_descriptive(df,group_column,value_column)

The Descriptives about "arrival_delay" grouped by "semester" are: 


semester arrival_delay                                         \
                           count         mean           std  min    25%   
0   first_semester       19809.0  3549.362058  10936.261175  0.0  268.0   
1  second_semester       13686.0  4447.012348  12967.029810  0.0  377.0   

                              
      50%      75%       max  
0   828.0  2312.00  305694.0  
1  1060.5  3057.75  287698.0

In [8]:
group_column = 'week_day_nbr'
value_column = 'carrier_delay'
category_descriptive(df,group_column,value_column)

The Descriptives about "carrier_delay" grouped by "week_day_nbr" are: 


week_day_nbr carrier_delay                                               \
                       count         mean          std  min    25%    50%   
0            1        6508.0  1224.240627  3972.634145  0.0   74.0  258.0   
1            2        5089.0  1730.804087  4905.977506  0.0  138.0  426.0   
2            3        5167.0  1768.296884  5340.364439  0.0  126.0  427.0   
3            4        3288.0  1558.945864  4729.427697  0.0  104.0  356.0   
4            5        6694.0  1576.535853  4800.904519  0.0   91.0  338.0   
5            6        3336.0  1423.761691  4303.243113  0.0  102.0  334.5   
6            7        3413.0  1925.492822  5440.817819  0.0  164.0  508.0   

                      
       75%       max  
0   854.25  101046.0  
1  1268.00  105806.0  
2  1286.00  111079.0  
3  1196.00  111645.0  
4  1134.75  119425.0  
5  1060.25   77072.0  
6  1478.00  102534.0

* Aggregation

* multiple agg

In [9]:
df.groupby("week_day").agg({
    "arrival_delay": ["sum", "mean", "std"],
    "total_delay": ["sum", "mean", "std"]
})

arrival_delay                            total_delay               \
                    sum         mean           std         sum         mean   
week_day                                                                      
Friday         26284089  3926.514640  12045.452851    52568171  7853.028234   
Monday         19358115  2974.510602   9619.780894    38716206  5949.017517   
Saturday       10954433  3283.702938   9866.740024    21908864  6567.405276   
Sunday         16669423  4884.096982  13833.597933    33338841  9768.192499   
Thursday       13282458  4039.677007  12285.204488    26564910  8079.352190   
Tuesday        22975799  4514.796424  12718.431977    45951598  9029.592847   
Wednesday      21646807  4189.434295  12398.034167    43293611  8378.868009   

                         
                    std  
week_day                 
Friday     24090.905702  
Monday     19239.562501  
Saturday   19733.480009  
Sunday     27667.195969  
Thursday   24570.408995  
Tuesday    25436.863955  
Wednesday  24796.068346

* Named Aggregation

In [10]:
df.groupby("week_day").agg(
    max_arrival_delay=pd.NamedAgg(column="arrival_delay", aggfunc="max"),
    variance_total_delay=pd.NamedAgg(column="total_delay", aggfunc="var")
).reset_index()

,week_day,max_arrival_delay,variance_total_delay
0,Friday,259825,5.803717e+08
1,Monday,259731,3.701608e+08
2,Saturday,215233,3.894102e+08
3,Sunday,281392,7.654737e+08
4,Thursday,287698,6.037050e+08
5,Tuesday,305694,6.470340e+08
6,Wednesday,273956,6.148450e+08


* transform

In [11]:
df.groupby('week_day')[['arrival_flights','arrival_del15']].transform(lambda x : x.mean())

,arrival_flights,arrival_del15
0,294.065765,46.174645
1,294.065765,46.174645
2,294.065765,46.174645
3,294.065765,46.174645
4,294.065765,46.174645
...,...,...
33490,310.428742,58.852133
33491,310.428742,58.852133
33492,310.428742,58.852133
33493,310.428742,58.852133


In [12]:
df.groupby(['year','week_day'])['weather_delay'].transform(lambda x: x.sum()/2)

0        307191.5
1        307191.5
2        307191.5
3        307191.5
4        307191.5
           ...   
33490    245553.5
33491    245553.5
33492    245553.5
33493    245553.5
33494    245553.5
Name: weather_delay, Length: 33495, dtype: float64

* apply

In [13]:
df.groupby(['year','week_day'])['weather_delay'].apply(sum).reset_index()

,year,week_day,weather_delay
0,2021,Friday,491107
1,2021,Monday,800023
2,2021,Saturday,354179
3,2021,Sunday,673066
4,2021,Thursday,899169
5,2021,Tuesday,644241
6,2021,Wednesday,714752
7,2022,Friday,888575
8,2022,Monday,614383
9,2022,Saturday,396204


* transform 

In [14]:
df[df.groupby(['week_day'])['weather_count'].transform('sum') > 12000]


,day,year,month,carrier,carrier_name,airport,airport_name,arrival_flights,arrival_del15,carrier_count,...,county_state,county,state,state_name,week_day_nbr,weekday_weekend,arrival_delay_bins,arrival_delay_bins_q,semester,total_delay
0,1,2022,8,9E,Endeavor Air Inc.,ABY,Southwest Georgia Regional,79,12.0,5.34,...,"Albany, GA",Albany,GA,Georgia,1,Weekday,small_delay,medium_delay_q,second_semester,1818
1,1,2022,8,9E,Endeavor Air Inc.,ACK,Nantucket Memorial,124,11.0,4.19,...,"Nantucket, MA",Nantucket,MA,Massachusetts,1,Weekday,small_delay,medium_delay_q,second_semester,1350
2,1,2022,8,9E,Endeavor Air Inc.,AEX,Alexandria International,62,4.0,3.18,...,"Alexandria, LA",Alexandria,LA,Louisiana,1,Weekday,small_delay,small_delay_q,second_semester,386
3,1,2022,8,9E,Endeavor Air Inc.,AGS,Augusta Regional at Bush Field,162,15.0,5.36,...,"Augusta, GA",Augusta,GA,Georgia,1,Weekday,small_delay,big_delay_q,second_semester,2220
4,1,2022,8,9E,Endeavor Air Inc.,ALB,Albany International,123,18.0,3.84,...,"Albany, NY",Albany,NY,New York,1,Weekday,small_delay,big_delay_q,second_semester,1968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33490,1,2021,1,YX,Republic Airline,TLH,Tallahassee International,131,8.0,4.34,...,"Tallahassee, FL",Tallahassee,FL,Florida,5,Weekday,small_delay,medium_delay_q,first_semester,1164
33491,1,2021,1,YX,Republic Airline,TPA,Tampa International,253,19.0,6.45,...,"Tampa, FL",Tampa,FL,Florida,5,Weekday,small_delay,big_delay_q,first_semester,2078
33492,1,2021,1,YX,Republic Airline,TUS,Tucson International,31,4.0,0.00,...,"Tucson, AZ",Tucson,AZ,Arizona,5,Weekday,small_delay,small_delay_q,first_semester,382
33493,1,2021,1,YX,Republic Airline,TVC,Cherry Capital,2,0.0,0.00,...,"Traverse City, MI",Traverse City,MI,Michigan,5,Weekday,small_delay,small_delay_q,first_semester,0


* filter

In [15]:
df.groupby(['week_day','airport']).filter(lambda g : g.weather_count.sum()>1000)

,day,year,month,carrier,carrier_name,airport,airport_name,arrival_flights,arrival_del15,carrier_count,...,county_state,county,state,state_name,week_day_nbr,weekday_weekend,arrival_delay_bins,arrival_delay_bins_q,semester,total_delay
118,1,2022,8,AA,American Airlines Inc.,DFW,Dallas/Fort Worth International,13051,2938.0,829.64,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,1,Weekday,very_big_delay,very_big_delay_q,second_semester,519462
230,1,2022,8,AS,Alaska Airlines Inc.,DFW,Dallas/Fort Worth International,178,52.0,19.84,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,1,Weekday,small_delay,big_delay_q,second_semester,4900
311,1,2022,8,B6,JetBlue Airways,DFW,Dallas/Fort Worth International,62,22.0,12.18,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,1,Weekday,small_delay,big_delay_q,second_semester,4138
391,1,2022,8,DL,Delta Air Lines Inc.,DFW,Dallas/Fort Worth International,921,181.0,73.68,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,1,Weekday,small_delay,very_big_delay_q,second_semester,27612
518,1,2022,8,F9,Frontier Airlines Inc.,DFW,Dallas/Fort Worth International,402,132.0,30.46,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,1,Weekday,small_delay,very_big_delay_q,second_semester,20458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32806,1,2021,1,OH,PSA Airlines Inc.,DFW,Dallas/Fort Worth International,29,9.0,3.09,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,5,Weekday,small_delay,big_delay_q,first_semester,2136
32923,1,2021,1,OO,SkyWest Airlines Inc.,DFW,Dallas/Fort Worth International,1953,400.0,207.00,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,5,Weekday,small_delay,very_big_delay_q,first_semester,83270
33152,1,2021,1,UA,United Air Lines Inc.,DFW,Dallas/Fort Worth International,166,23.0,5.35,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,5,Weekday,small_delay,big_delay_q,first_semester,2452
33337,1,2021,1,YV,Mesa Airlines Inc.,DFW,Dallas/Fort Worth International,1985,468.0,43.96,...,"Dallas/Fort Worth, TX",Dallas/Fort Worth,TX,Texas,5,Weekday,small_delay,very_big_delay_q,first_semester,62728


In [16]:
df.head()

,day,year,month,carrier,carrier_name,airport,airport_name,arrival_flights,arrival_del15,carrier_count,...,county_state,county,state,state_name,week_day_nbr,weekday_weekend,arrival_delay_bins,arrival_delay_bins_q,semester,total_delay
0,1,2022,8,9E,Endeavor Air Inc.,ABY,Southwest Georgia Regional,79,12.0,5.34,...,"Albany, GA",Albany,GA,Georgia,1,Weekday,small_delay,medium_delay_q,second_semester,1818
1,1,2022,8,9E,Endeavor Air Inc.,ACK,Nantucket Memorial,124,11.0,4.19,...,"Nantucket, MA",Nantucket,MA,Massachusetts,1,Weekday,small_delay,medium_delay_q,second_semester,1350
2,1,2022,8,9E,Endeavor Air Inc.,AEX,Alexandria International,62,4.0,3.18,...,"Alexandria, LA",Alexandria,LA,Louisiana,1,Weekday,small_delay,small_delay_q,second_semester,386
3,1,2022,8,9E,Endeavor Air Inc.,AGS,Augusta Regional at Bush Field,162,15.0,5.36,...,"Augusta, GA",Augusta,GA,Georgia,1,Weekday,small_delay,big_delay_q,second_semester,2220
4,1,2022,8,9E,Endeavor Air Inc.,ALB,Albany International,123,18.0,3.84,...,"Albany, NY",Albany,NY,New York,1,Weekday,small_delay,big_delay_q,second_semester,1968


* Custom Function for aggregation

In [17]:
# https://stackoverflow.com/questions/51631096/python-pandas-aggregation-with-condition

def my_aggregations(group,iairport='AGS',istate='FL'):
    
    a = group['airport'].size
    b = group.loc[group.airport==iairport].shape[0]
    c = b/a
    
    d = group.loc[group.airport==iairport].arrival_delay.mean()
    e = group.loc[(group.airport==iairport) & (group.state==istate)].arrival_delay.mean()
    
    f = group.loc[group.arrival_delay.notnull()].shape[0]
    g = group.query('county==county').shape[0]
    
    h = np.where(c>0.6,group.total_delay.mean(),group.arrival_delay.mean())
    
    cols = ['count_obs','count_AGS','AGS_ratio',
            'avg_AGS_arrival_delay','avg_FL_AGS_arrival_delay',
            'not_null_arirval_delay','not_null_FL',
            'kpi_1']
    
    return pd.Series([a,b,c,d,e,f,g,h], index=cols)

In [18]:
dfw = df.groupby('year').apply(my_aggregations).reset_index().round(2)
dfw

,year,count_obs,count_AGS,AGS_ratio,avg_AGS_arrival_delay,avg_FL_AGS_arrival_delay,not_null_arirval_delay,not_null_FL,kpi_1
0,2021,19918,43,0.0,593.35,NaN,19918,19918,3346.0571844562705
1,2022,13577,30,0.0,856.93,NaN,13577,13577,4752.475289091846


* get_group

In [19]:
df_grouped = df.groupby('week_day')

In [20]:
df_grouped.get_group('Wednesday')

,day,year,month,carrier,carrier_name,airport,airport_name,arrival_flights,arrival_del15,carrier_count,...,county_state,county,state,state_name,week_day_nbr,weekday_weekend,arrival_delay_bins,arrival_delay_bins_q,semester,total_delay
3372,1,2022,6,9E,Endeavor Air Inc.,ABE,Lehigh Valley International,133,14.0,9.28,...,"Allentown/Bethlehem/Easton, PA",Allentown/Bethlehem/Easton,PA,Pennsylvania,3,Weekday,small_delay,medium_delay_q,first_semester,1706
3373,1,2022,6,9E,Endeavor Air Inc.,ABY,Southwest Georgia Regional,89,15.0,7.24,...,"Albany, GA",Albany,GA,Georgia,3,Weekday,small_delay,medium_delay_q,first_semester,1706
3374,1,2022,6,9E,Endeavor Air Inc.,ACK,Nantucket Memorial,120,12.0,5.25,...,"Nantucket, MA",Nantucket,MA,Massachusetts,3,Weekday,small_delay,big_delay_q,first_semester,4090
3375,1,2022,6,9E,Endeavor Air Inc.,AEX,Alexandria International,78,15.0,4.03,...,"Alexandria, LA",Alexandria,LA,Louisiana,3,Weekday,small_delay,big_delay_q,first_semester,3294
3376,1,2022,6,9E,Endeavor Air Inc.,AGS,Augusta Regional at Bush Field,176,22.0,12.78,...,"Augusta, GA",Augusta,GA,Georgia,3,Weekday,small_delay,big_delay_q,first_semester,2228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20466,1,2021,9,YX,Republic Airline,TUS,Tucson International,30,0.0,0.00,...,"Tucson, AZ",Tucson,AZ,Arizona,3,Weekday,small_delay,small_delay_q,second_semester,0
20467,1,2021,9,YX,Republic Airline,TVC,Cherry Capital,76,8.0,4.00,...,"Traverse City, MI",Traverse City,MI,Michigan,3,Weekday,small_delay,medium_delay_q,second_semester,976
20468,1,2021,9,YX,Republic Airline,TYS,McGhee Tyson,2,1.0,0.43,...,"Knoxville, TN",Knoxville,TN,Tennessee,3,Weekday,small_delay,small_delay_q,second_semester,98
20469,1,2021,9,YX,Republic Airline,VPS,Eglin AFB Destin Fort Walton Beach,9,0.0,0.00,...,"Valparaiso, FL",Valparaiso,FL,Florida,3,Weekday,small_delay,small_delay_q,second_semester,0


* Wide to Long

In [21]:
# melt
id_vars_long =['day', 'year', 'month', 'carrier','airport_name']
df_long = pd.melt(df, id_vars=id_vars_long, var_name='variable', value_name='value', ignore_index=True).sort_values(by = ['airport_name']).drop_duplicates().reset_index(drop = True)
df_long.head()

,day,year,month,carrier,airport_name,variable,value
0,1,2021,10,OO,Aberdeen Regional,late_aircraft_count,0.0
1,1,2021,2,OO,Aberdeen Regional,date,2021-02-01 00:00:00
2,1,2021,10,OO,Aberdeen Regional,arrival_diverted,0.0
3,1,2021,7,OO,Aberdeen Regional,arrival_flights,62
4,1,2021,8,OO,Aberdeen Regional,arrival_cancelled,0.0


In [22]:
# wide_to_long
df_example_to_long = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
                   "A1980" : {0 : "d", 1 : "e", 2 : "f"},
                   "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
                   "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
                   "X"     : dict(zip(range(3), np.random.randn(3)))
                  })
df_example_to_long["id"] = df_example_to_long.index
df_example_to_long

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-0.720081,0
1,b,e,1.2,1.3,0.932329,1
2,c,f,0.7,0.1,-0.620518,2


In [23]:
pd.wide_to_long(df_example_to_long, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-0.720081,a,2.5
1,1970,0.932329,b,1.2
2,1970,-0.620518,c,0.7
0,1980,-0.720081,d,3.2
1,1980,0.932329,e,1.3
2,1980,-0.620518,f,0.1


* Long to Wide

In [24]:
index_vars_long =['day', 'year', 'month', 'carrier','airport_name']
pd.pivot(df_long, index = index_vars_long,columns = 'variable', values = 'value').sort_index(level=[1,0]).reset_index()

variable,day,year,month,carrier,airport_name,airport,arrival_cancelled,arrival_del15,arrival_delay,arrival_delay_bins,...,security_delay,semester,state,state_name,total_delay,weather_count,weather_delay,week_day,week_day_nbr,weekday_weekend
0,1,2021,1,9E,Albany International,ALB,0.0,10.0,430,small_delay,...,0,first_semester,NY,New York,860,0.0,0,Friday,5,Weekday
1,1,2021,1,9E,Albert J Ellis,OAJ,0.0,8.0,2054,small_delay,...,0,first_semester,NC,North Carolina,4108,3.0,1875,Friday,5,Weekday
2,1,2021,1,9E,Alexandria International,AEX,0.0,7.0,969,small_delay,...,0,first_semester,LA,Louisiana,1938,1.0,541,Friday,5,Weekday
3,1,2021,1,9E,Appleton International,ATW,0.0,17.0,1928,small_delay,...,0,first_semester,WI,Wisconsin,3856,2.0,922,Friday,5,Weekday
4,1,2021,1,9E,Asheville Regional,AVL,0.0,5.0,740,small_delay,...,0,first_semester,NC,North Carolina,1480,1.97,611,Friday,5,Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33490,1,2022,8,YX,Tulsa International,TUL,1.0,8.0,305,small_delay,...,0,second_semester,OK,Oklahoma,610,0.54,30,Monday,1,Weekday
33491,1,2022,8,YX,Washington Dulles International,IAD,29.0,77.0,6770,small_delay,...,0,second_semester,DC,District Of Columbia,13540,5.6,369,Monday,1,Weekday
33492,1,2022,8,YX,Will Rogers World,OKC,7.0,18.0,768,small_delay,...,0,second_semester,OK,Oklahoma,1536,0.0,0,Monday,1,Weekday
33493,1,2022,8,YX,Wilmington International,ILM,10.0,27.0,1369,small_delay,...,0,second_semester,NC,North Carolina,2738,1.33,36,Monday,1,Weekday


In [25]:
# Pivot Table
df_example = pd.DataFrame({
    "name": ["Kevin", "Kevin", "Kevin", "Mary", "Mary", "Mary", "Bob", "Bob", "Bob"],
    "course": ["A", "B", "C", "A", "B", "C","A", "B", "C"],
    "score": [66, 75, 80, 70, 77, 81, 80, 85, 90],
    "sex": ["M", "M", "M", "F", "F", "F", "M", "M", "M"],
})
df_example

,name,course,score,sex
0,Kevin,A,66,M
1,Kevin,B,75,M
2,Kevin,C,80,M
3,Mary,A,70,F
4,Mary,B,77,F
5,Mary,C,81,F
6,Bob,A,80,M
7,Bob,B,85,M
8,Bob,C,90,M


* pivot_table can handle duplicate values for a pivoted index/column pair.
* Specifically, you can give pivot_table a list of aggregation functions using keyword argument aggfunc.
* The default aggfunc of pivot_table is numpy.mean.
* If we want to count the number of courses and the mean of scores for each student, we can use the code below

In [26]:
pd.pivot_table(df_example, index="name", values=["course", "score"], aggfunc={"course": "count", "score": np.mean}) 

,course,score
name,,
Bob,3,85.000000
Kevin,3,73.666667
Mary,3,76.000000


* stack

In [27]:
# Simple Level
df_single_level = pd.DataFrame(
    [['Mostly cloudy', 10], ['Sunny', 12]],
    index=['London', 'Oxford'],
    columns=['Weather', 'Wind']
)
print('Dataframe: \n')
display(df_single_level)
print('\n')
print('Stacked Dataframe: \n')
display(df_single_level.stack())

Dataframe: 



,Weather,Wind
London,Mostly cloudy,10
Oxford,Sunny,12




Stacked Dataframe: 



London  Weather    Mostly cloudy
        Wind                  10
Oxford  Weather            Sunny
        Wind                  12
dtype: object

In [28]:
# Multiple Levels
multi_col_1 = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Wind', 'm/s')]
)
df_multi_level_1 = pd.DataFrame(
    [[13, 5.5], [19, 8.5]],
    index=['London', 'Oxford'],
    columns=multi_col_1
)

print('Dataframe: \n')
display(df_multi_level_1)
print('\n')
print('Stacked Dataframe: \n')
display(df_multi_level_1.stack())

Dataframe: 



Wind     
        mph  m/s
London   13  5.5
Oxford   19  8.5



Stacked Dataframe: 



Wind
London m/s   5.5
       mph  13.0
Oxford m/s   8.5
       mph  19.0

* unstack

In [29]:
# Simple case
index = pd.MultiIndex.from_tuples([
  ('Oxford', 'Temperature'), 
  ('Oxford', 'Wind'),
  ('London', 'Temperature'), 
  ('London', 'Wind')
])
s = pd.Series([1,2,3,4], index=index)


print('Series: \n')
display(s)
print('\n')
print('Unstacked Series: \n')
display(s.unstack())
print('Unstack a specific level')
display(s.unstack(level=0))


Series: 



Oxford  Temperature    1
        Wind           2
London  Temperature    3
        Wind           4
dtype: int64



Unstacked Series: 



,Temperature,Wind
London,3,4
Oxford,1,2


Unstack a specific level


,London,Oxford
Temperature,3,1
Wind,4,2


In [30]:
#  unstack: more levels
index = pd.MultiIndex.from_tuples([
  ('Oxford', 'Weather', '01-01-2022'), 
  ('Oxford', 'Temperature', '01-01-2022'), 
  ('Oxford', 'Weather', '02-01-2022'),
  ('Oxford', 'Temperature', '02-01-2022'),
  ('London', 'Weather', '01-01-2022'), 
  ('London', 'Temperature', '01-01-2022'),
  ('London', 'Weather', '02-01-2022'),
  ('London', 'Temperature', '02-01-2022'),
])
s = pd.Series(
  ['Sunny', 10, 'Shower', 7, 'Shower', 5, 'Sunny', 8], 
  index=index
)

print('Series: \n')
display(s)

print('\n')
print('Unstacked Series (Method Chaining): \n')
display(s.unstack().unstack())
# print('Unstacked Series (Pass a list): \n')
display(s.unstack([2,1]))

Series: 



Oxford  Weather      01-01-2022     Sunny
        Temperature  01-01-2022        10
        Weather      02-01-2022    Shower
        Temperature  02-01-2022         7
London  Weather      01-01-2022    Shower
        Temperature  01-01-2022         5
        Weather      02-01-2022     Sunny
        Temperature  02-01-2022         8
dtype: object



Unstacked Series (Method Chaining): 



01-01-2022          02-01-2022        
       Temperature Weather Temperature Weather
London           5  Shower           8   Sunny
Oxford          10   Sunny           7  Shower

01-01-2022             02-01-2022            
          Weather Temperature    Weather Temperature
London     Shower           5      Sunny           8
Oxford      Sunny          10     Shower           7

### Confusion Matrix
#### pd.crosstab()

In [31]:
pd.crosstab(df['arrival_delay_bins'],df['quarter']).reset_index()

quarter,arrival_delay_bins,1,2,3,4
0,small_delay,9690,10023,8460,5128
1,medium_delay,35,49,61,23
2,big_delay,1,9,7,2
3,very_big_delay,0,2,5,0


In [32]:
pd.crosstab(df['arrival_delay_bins'],df['week_day_nbr']).reset_index()

week_day_nbr,arrival_delay_bins,1,2,3,4,5,6,7
0,small_delay,6486,5051,5134,3266,6654,3320,3390
1,medium_delay,19,35,29,19,34,15,17
2,big_delay,2,2,3,2,4,1,5
3,very_big_delay,1,1,1,1,2,0,1


In [33]:
pd.crosstab(df['arrival_delay_bins'],df['weekday_weekend']).reset_index()

weekday_weekend,arrival_delay_bins,Weekday,Weekend
0,small_delay,26591,6710
1,medium_delay,136,32
2,big_delay,13,6
3,very_big_delay,6,1


* Output

In [34]:
df.to_pickle("df_2.pkl")